# **Predictive Default Risk Assessor V.01**

# TODO

* Base model 
* Comparison
* Specialised
* For small entities - Examples?
* Backtest - All sectors 
* Understanding the model across all sectors/industries
* Any markets - consumer goods, industries
* UI last step after backtesting

In [31]:
import pandas as pd
import numpy as np

In [32]:
model_inputs = {
    "profitability": {
        "class_weight": 0.30,
        "weights": [1.0], 
        "metrics": {
            "oper_margin": {
                "lower_is_better": False,
                "thresholds": [
                    (50, 45),
                     (45, 40),
                     (40, 35),
                     (35, 30),
                     (30, 25),
                     (25, 20),
                     (20, 15),
                     (15, 10),
                     (5, 0)]
            }
        },
    },
    "leverage_coverage": {
        "class_weight": 0.55,
        "weights": [0.3, 0.2, 0.5],
        "metrics": {
            "tot_debt_to_tot_eqy": {
                "lower_is_better": True,
                "thresholds": [
                     (float("-inf"), 2.0),
                     (2.0, 16.0),
                     (16.0, 24.0),
                     (24.0, 33.0),
                     (33.0, 43.0),
                     (43.0, 54.0),
                     (54.0, 68.0),
                     (68.0, 94.0),
                     (94.0, float("inf")),
                ],
            },
            "tot_debt_to_ebitda": {
                "lower_is_better": True,
                "thresholds": [
                    (float("-inf"), 0.09),
                    (0.09, 0.49),
                    (0.49, 0.9),
                    (0.9, 1.36),
                    (1.36, 1.68),
                    (1.68, 2.26),
                    (2.26, 3.27),
                    (3.27, 4.4),
                    (4.4, float("inf")),
                ],
            },
            "ebitda_to_tot_int_exp": {
                "lower_is_better": False,
                "thresholds": [
                    (25, float("inf")),
                    (20, 25),
                    (15, 20),
                    (10, 15),
                    (5, 10),
                    (3, 5),
                    (1, 3),
                    (0, 1),
                    (float("-inf"), 0),
                ],
            },
        },
    },
    "efficiency": {
        "class_weight": 0.15,
        "weights": [0.3, 0.7],
        "metrics": {
            "return_on_asset": {
                "lower_is_better": False,
                "thresholds": [
                    (0.15, float("inf")),
                    (0.10, 0.15),
                    (0.08, 0.10),
                    (0.06, 0.08),
                    (0.04, 0.06),
                    (0.02, 0.04),
                    (0.00, 0.02),
                    (-0.02, 0.00),
                    (float("-inf"), -0.02)
                ],
            },
            "asset_turnover": {
                "lower_is_better": False,
                "thresholds": [
                    (4.0, float("inf")),
                    (3.0, 4.0),
                    (2.0, 3.0),
                    (1.5, 2.0),
                    (1.0, 1.5),
                    (0.75, 1.0),
                    (0.5, 0.75),
                    (0.25, 0.5),
                    (float("-inf"), 0.25)
                ],
            },
        },
    },
}

In [33]:
model_metrics = [
    'oper_margin', 'tot_debt_to_tot_eqy', 'tot_debt_to_ebitda',
    'ebitda_to_tot_int_exp', 'return_on_asset', 'asset_turnover',
]

In [34]:
class CreditRatingCalculator:
    def __init__(self, metrics):
        self.metrics = metrics
        
    def _calculate_metric_score(self, metric, thresholds, inverse):
        for score, (lower, upper) in enumerate(thresholds, start=1):
            if (inverse and metric <= upper) or (not inverse and metric >= lower):
                return score
        return len(thresholds) // 2 # else return the middle score

    def _calculate_category_score(self, category_metrics, ratios):
        total_weighted_score = 0

        for metric, weight in zip(
            category_metrics["metrics"].items(), category_metrics["weights"]
        ):
            metric_name, metric_data = metric
            value = ratios[metric_name]
            score = self._calculate_metric_score(
                value, metric_data["thresholds"], metric_data["lower_is_better"]
            )
            total_weighted_score += score * weight

        return total_weighted_score

    def _calculate_scores(self, ratios):
        scores = {}
        for category, category_data in self.metrics.items():
            category_score = self._calculate_category_score(category_data, ratios)
            scores[category] = category_score
        return scores

    def _calculate_weighted_score(self, scores):
        weights = {
            category: category_data["class_weight"]
            for category, category_data in self.metrics.items()
        }
        return sum(scores[category] * weight for category, weight in weights.items())

    def _determine_credit_rating(self, weighted_score):
        credit_ratings = {
            "Aaa": 2.5,
            "Aa": 3.5,
            "A": 4.5,
            "Baa": 5.5,
            "Ba": 6.5,
            "B": 7.5,
            "Caa": 8.5,
            "Ca": 9.5,
            "C": float("inf")
        }
        return next(rating for rating, threshold in credit_ratings.items() if weighted_score <= threshold)

    def calculate_credit_rating(self, ratios):
        self.scores = self._calculate_scores(ratios)
        self.credit_score = self._calculate_weighted_score(self.scores)
        self.credit_rating = self._determine_credit_rating(self.credit_score)

In [35]:
def determine_credit_rating(weighted_score):
    credit_ratings = [
        (2.5, "Aaa"),
        (3.5, "Aa"),
        (4.5, "A"),
        (5.5, "Baa"),
        (6.5, "Ba"),
        (7.5, "B"),
        (8.5, "Caa"),
        (9.5, "Ca"),
        (10, "C"),
    ]

    for threshold, rating in credit_ratings:
        if weighted_score < threshold:
            return rating

In [56]:
features = pd.read_excel("dataset/features.xlsx", index_col=0)
targets = pd.read_excel("dataset/target.xlsx", index_col=0)
features.columns = features.columns.str.lower()

In [37]:
features[model_metrics].describe()

,oper_margin,tot_debt_to_tot_eqy,tot_debt_to_ebitda,ebitda_to_tot_int_exp,return_on_asset,asset_turnover
count,70.000000,70.000000,61.000000,51.000000,70.000000,70.000000
mean,18.470457,181.766303,4.716249,5.665463,3.134256,0.606143
std,22.334470,437.735333,7.996142,5.156308,6.785515,0.663520
min,-57.669600,4.368500,0.138700,-0.592700,-24.402700,0.030300
25%,5.596050,47.506775,1.884500,1.794250,0.830700,0.188175
50%,16.206650,78.141850,3.366400,3.520000,2.304850,0.485300
75%,31.520875,149.426875,5.067300,8.350250,6.090200,0.688875
max,64.345000,3272.663800,61.124000,24.311400,21.310600,3.852600


In [38]:
import joblib
import pandas as pd

study = {}
studies = {}

#features = features.dropna()
for company in features.index:
    ratios = features.loc[company][model_metrics].to_dict()    
    model = CreditRatingCalculator(model_inputs)
    model.calculate_credit_rating(ratios)
    
    credit_score = model.credit_score
    credit_rating = model.credit_rating
    
    study[company] = {
        "model_class_scores": model.scores, 
        "model_inputs": ratios, 
        "model_credit_ratings": credit_rating, 
        "model_credit_scores": float(credit_score),
    }
study = pd.DataFrame(study).T


In [39]:
from sklearn.metrics import mean_squared_error

In [40]:
mean_squared_error(study['model_credit_scores'], targets.loc[features.index]['numeric_rating'])

1.2683428571428572

In [49]:
study['target'] = targets.loc[features.index]['numeric_rating']

In [53]:
study['rating'] = targets.loc[features.index]["rating"]

In [55]:
features

,asset_turnover,bs_lt_borrow,bs_st_borrow,bs_total_liabilities,bs_tot_asset,cfo_to_tot_debt,ebitda,ebitda_to_interest_expn,ebitda_to_revenue,ebitda_to_tot_int_exp,...,return_on_cap,return_on_inv_capital,sales_growth,sales_rev_turn,short_and_long_term_debt,total_equity,tot_debt_to_ebitda,tot_debt_to_tot_asset,tot_debt_to_tot_cap,tot_debt_to_tot_eqy
AGL SJ Equity,0.4577,2.770832e+05,31777.2720,6.378648e+05,1.215280e+06,0.3649,121616.1358,4.7608,21.4962,3.4697,...,3.8440,3.3318,-12.7171,5.657577e+05,3.088605e+05,5.774149e+05,2.5667,25.4148,34.8493,53.4902
ANG SJ Equity,0.5640,3.889976e+04,5113.5840,8.099552e+04,1.492984e+05,0.4531,18864.5579,9.6807,25.5943,9.6807,...,-12.4486,-2.1459,1.7996,8.457203e+04,4.401335e+04,6.830287e+04,1.8845,29.4801,39.1870,64.4385
ANH SJ Equity,0.2747,1.354406e+06,73124.2512,2.313239e+06,4.005763e+06,0.1697,357280.8529,5.0252,32.5985,4.9105,...,5.8489,5.9292,2.7585,1.096003e+06,1.427530e+06,1.692523e+06,4.0381,35.6369,45.7534,84.3433
BAW SJ Equity,0.8784,7.705000e+03,4142.0000,3.130300e+04,4.812200e+04,0.1198,6624.0000,4.1374,14.7108,4.1374,...,10.6581,9.6124,14.3336,4.502800e+04,1.184700e+04,1.681900e+04,1.7885,24.6187,41.3277,70.4382
BHG SJ Equity,0.5479,2.852730e+05,134871.0498,9.921380e+05,1.904628e+06,0.8369,486944.0718,32.0082,50.9114,24.3114,...,21.8961,20.3470,-17.3293,9.564535e+05,4.201441e+05,9.124902e+05,0.8155,22.0591,31.5273,46.0437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UGPA3 BS Equity,3.3751,1.098727e+04,2304.6800,2.422215e+04,3.825197e+04,0.2896,5720.7110,3.5200,4.5385,3.5200,...,13.8756,12.2053,-12.2436,1.260487e+05,1.329195e+04,1.402983e+04,2.3235,34.7484,48.6497,94.7407
USIM5 BS Equity,0.6896,5.855413e+03,167.9600,1.361231e+04,4.016175e+04,0.7852,1592.4070,2.3043,5.7616,2.3043,...,6.4728,6.5389,-14.8817,2.763835e+04,6.023373e+03,2.654944e+04,3.7826,14.9978,18.4920,22.6874
VALE3 BS Equity,0.4576,6.246400e+04,4940.0000,2.576590e+05,4.559840e+05,0.9927,85917.0000,14.8800,41.2931,14.6366,...,16.9916,23.7007,-8.1419,2.080660e+05,6.740400e+04,1.983250e+05,0.7845,14.7821,25.3657,33.9866
VAMO3 BS Equity,0.3247,1.083538e+04,881.6250,1.607353e+04,2.080883e+04,-0.2292,2668.1330,1.7422,43.8442,1.7422,...,11.6270,11.4471,23.8534,6.085482e+03,1.171701e+04,4.735295e+03,4.3915,56.3079,71.2180,247.4399


In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel("world_equities_universe_credit.xlsx")

In [11]:
moodys = df["rtg_moody_long_term"].dropna().index.tolist()
sp = df['rtg_sp_lt_lc_issuer_credit'].dropna().index.tolist()

In [14]:
vaild = list(set(moodys + sp))

In [19]:
data = df.loc[vaild]

In [22]:
data = data[data.gics_sector_name != "Financials"]

In [30]:
data[data.id.str.endswith("SJ Equity")]

,id,name,gics_sector_name,gics_industry_name,rsk_bb_issuer_default,bb_5y_default_prob,rtg_moody_long_term,rtg_moody_long_term_date,rtg_sp_lt_lc_issuer_credit,rtg_sp_lt_lc_iss_cred_rtg_dt,country_full_name,book_val_per_sh,bs_tot_asset
188,GFI SJ Equity,Gold Fields Ltd,Materials,Metals & Mining,IG7,0.021680,Baa3,4/6/2022,BBB-,4/28/2022,SOUTH AFRICA,5.009396,8.226300e+09
521,SOL SJ Equity,Sasol Ltd,Materials,Chemicals,IG10,0.038086,Ba1,11/28/2023,BB+,10/28/2022,SOUTH AFRICA,309.864959,4.271380e+11
1569,ANG SJ Equity,Anglogold Ashanti Plc,Materials,Metals & Mining,IG7,0.023611,Baa3,9/26/2023,BB+,4/9/2024,BRITAIN,8.865255,8.175000e+09
1856,MTN SJ Equity,MTN Group Ltd,Communication Services,Wireless Telecommunication Services,IG8,0.023938,Ba2,4/6/2022,BB-,5/12/2020,SOUTH AFRICA,77.065739,4.347140e+11
1991,SSW SJ Equity,Sibanye Stillwater Ltd,Materials,Metals & Mining,HY2,0.077816,Ba2,4/27/2022,BB-,12/12/2023,SOUTH AFRICA,17.215632,1.429410e+11
2164,BVT SJ Equity,Bidvest Group Ltd,Industrials,Industrial Conglomerates,IG4,0.011689,Ba2,4/6/2022,NaN,NaN,SOUTH AFRICA,100.080076,1.111574e+11
2514,PPH SJ Equity,Pepkor Holdings Ltd,Consumer Discretionary,Specialty Retail,IG5,0.013262,Ba2,4/14/2022,NaN,NaN,SOUTH AFRICA,16.046087,1.087400e+11


In [35]:
df = pd.read_excel('resources/metrics.xlsx', index_col=0, parse_dates=True, header=[0, 1])

ABG SJ Equity                                                    \
Dates      RETURN_ON_ASSET SALES_GROWTH GROSS_MARGIN RETURN_COM_EQY PE_RATIO   
2000-01-31             NaN          NaN          NaN            NaN      NaN   
2000-02-29             NaN          NaN          NaN            NaN      NaN   
2000-03-31             NaN          NaN          NaN            NaN   7.6115   
2000-04-28             NaN          NaN          NaN            NaN   7.6433   
2000-05-31             NaN          NaN          NaN            NaN   8.1210   
...                    ...          ...          ...            ...      ...   
2023-09-29          1.2442      36.8491          NaN        16.4314   7.0936   
2023-10-31          1.2442      36.8491          NaN        16.4314   6.8944   
2023-11-30          1.2442      36.8491          NaN        16.4314   7.0502   
2023-12-29          1.2442      36.8491          NaN        16.4314   6.6425   
2024-01-31          1.2442      36.8491          NaN        16.4314   6.6461   

                                                                         \
Dates      TOT_DEBT_TO_EBITDA CFO_TO_TOT_DEBT BS_TOT_ASSET  CUR_MKT_CAP   
2000-01-31                NaN             NaN          NaN   16718.7538   
2000-02-29                NaN             NaN          NaN   15754.2103   
2000-03-31                NaN          0.2269     177234.0   15368.3929   
2000-04-28                NaN          0.2269     177234.0   15432.6958   
2000-05-31                NaN          0.2269     177234.0   16537.9484   
...                       ...             ...          ...          ...   
2023-09-29                NaN          0.1456    1896564.0  156363.9147   
2023-10-31                NaN          0.1456    1896564.0  151972.5240   
2023-11-30                NaN          0.1456    1896564.0  155406.9314   
2023-12-29                NaN          0.1456    1896564.0  146418.4434   
2024-01-31                NaN          0.1456    1896564.0  146498.9374   

                                ...   WHL SJ Equity                            \
Dates      LQA_LIQUIDITY_SCORE  ... RETURN_ON_ASSET SALES_GROWTH GROSS_MARGIN   
2000-01-31                 NaN  ...             NaN          NaN          NaN   
2000-02-29                 NaN  ...             NaN          NaN          NaN   
2000-03-31                 NaN  ...             NaN          NaN          NaN   
2000-04-28                 NaN  ...             NaN          NaN          NaN   
2000-05-31                 NaN  ...             NaN          NaN          NaN   
...                        ...  ...             ...          ...          ...   
2023-09-29                 NaN  ...          4.9244     -19.7116      35.1470   
2023-10-31                 NaN  ...          6.6379     -14.2868      35.8984   
2023-11-30                 NaN  ...          6.6379     -14.2868      35.8984   
2023-12-29                 NaN  ...          6.6379     -14.2868      35.8984   
2024-01-31                 NaN  ...          6.6379     -14.2868      35.8984   

                                                                       \
Dates      RETURN_COM_EQY PE_RATIO TOT_DEBT_TO_EBITDA CFO_TO_TOT_DEBT   
2000-01-31            NaN      NaN                NaN             NaN   
2000-02-29            NaN      NaN                NaN             NaN   
2000-03-31            NaN      NaN                NaN             NaN   
2000-04-28            NaN      NaN                NaN             NaN   
2000-05-31            NaN      NaN                NaN             NaN   
...                   ...      ...                ...             ...   
2023-09-29        32.0751  16.8953             1.0195          0.1650   
2023-10-31        34.8075  16.8953             1.0195          0.2103   
2023-11-30        34.8075  14.8265             1.0195          0.2103   
2023-12-29        34.8075  14.4493             1.0195          0.2103   
2024-01-31        34.8075  15.3047             1.0195          0.2